# Загрузка Pandas и очистка данных

In [46]:
import pandas as pd
import re
import numpy as np
import math

In [47]:
df = pd.read_csv('main_task.xls')

In [48]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [27]:
# Добавим новый признак(cost_living), а именно коэффициент стоимости жизни в городе
# Вдруг он будет оказывать тоже какое-либо, пусть и незначительное влияние
conditions = [(df['City']=='London'),
              (df['City']=='Paris'),
              (df['City']=='Madrid'),
              (df['City']=='Barcelona'),
              (df['City']=='Berlin'),
              (df['City']=='Milan'),
              (df['City']=='Rome'),
              (df['City']=='Prague'),
              (df['City']=='Lisbon'),
              (df['City']=='Vienna'),
              (df['City']=='Amsterdam'),
              (df['City']=='Brussels'),
              (df['City']=='Hamburg'),
              (df['City']=='Munich'),
              (df['City']=='Lyon'),
              (df['City']=='Stockholm'),
              (df['City']=='Budapest'),
              (df['City']=='Warsaw'),
              (df['City']=='Dublin'),
              (df['City']=='Copenhagen'),
              (df['City']=='Athens'),
              (df['City']=='Edinburgh'),
              (df['City']=='Zurich'),
              (df['City']=='Oporto'),
              (df['City']=='Geneva'),
              (df['City']=='Krakow'),
              (df['City']=='Oslo'),
              (df['City']=='Helsinki'),
              (df['City']=='Bratislava'),
              (df['City']=='Luxembourg'),
              (df['City']=='Ljubljana')
              ]

values = [85.86,
          90.87,
          62.20,
          63.50,
          69.41,
          78.81,
          75.59,
          50.84,
          53.18,
          69.41,
          77.67,
          75.66,
          72.23,
          79.74,
          78.54,
          82.04,
          43.24,
          43.28,
          82.59,
          93.91,
          57.38,
          75.78,
          127.07,
          51.28,
          120.03,
          39.74,
          103.85,
          77.95,
          49.49,
          85.88,
          73.90]

population = [8908081, 10784000, 3155360, 1593085, 3644826,
             1331586, 2870500, 1301132, 505526, 1911191,
             825080, 176545, 1718187, 1364920, 542684,
             1253309, 1744665, 1720398, 506246, 602481, 664046,
             482005, 428737, 214349, 198979, 766683, 634293,
             631695, 424428, 613894, 279631]
df['c_living'] = np.select(conditions, values)
df['population'] = np.select(conditions, population)
# Сразу в коментарии укажу что после добавление MAE изменился на ~0.001
# Влияние  оказалось незначительным, но из деталей можно собрать пазл, да и работа уже проделена, оставлю

In [49]:
# Также создадим колонку является ли город столицей или нет

def capital(City):
    
    capital = ['Paris', 'Stockholm', 'London', 'Berlin',
           'Bratislava', 'Vienna', 'Rome', 'Madrid',
           'Dublin', 'Brussels', 'Warsaw', 'Budapest',
           'Copenhagen', 'Amsterdam', 'Lisbon', 'Prague',
           'Oslo', 'Helsinki', 'Edinburgh', 'Ljubljana',
           'Athens', 'Luxembourg']
    
    if City in capital:
        return 1
    else:
        return 0


df['Capital'] = df['City'].apply(capital)

In [50]:
# Сразу же создадим dummy признак по городоам
df = pd.concat([df.drop('City', axis=1), pd.get_dummies(df['City'])], axis=1)

# И заменим NaN значение в признаке Number of Reviews
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

In [51]:
#Заменим также значения в колокне Price Range

def price(x):
    if x == '$':
        return 1
    elif x == '$$ - $$$':
        return 2
    elif x == '$$$$':
        return 3

df['Price Range'] = df['Price Range'].apply(price)
df['Price Range'] = df['Price Range'].fillna(2)

In [52]:
# Напимшем функцию по обработке ячеек, где есть не одно значение

def splitting_row(str_val):
    if pd.isna(str_val):
        return ["Unknown"]
    str_val = str_val.strip('[]')
    str_val = str_val.replace("\'",'')
    str_val = str_val.split(",")
    return str_val

def counter(row):
    for i in row:
        if i=='Unknown':
            return 1
        else:
            return len(i)

In [53]:
# Применим функцию

df['Cuisine Style'] = df['Cuisine Style'].apply(splitting_row)

# И сразу же приведем колонку в порядок, разбив значения и приведя к общему виду

#df = df.explode('Cuisine Style')
#df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x.lower().lstrip())

df['Cuisine Style']= df['Cuisine Style'].apply(counter)

In [54]:
# И также создадим dummy признак по кухням

#df = pd.concat([df.drop('Cuisine Style', axis=1), pd.get_dummies(df['Cuisine Style'])], axis=1)

In [55]:
# Теперь поработаем над датами через паттерн

pattern = re.compile('\d+\/\d+\/\d+')

df['Reviews'] = df['Reviews'].apply(pattern.findall)

df['Reviews'] =df['Reviews'].apply(lambda x:[x[-2], x[-1]] if len(x)>=3
                                  else x)

# Так как для модели нам нужны исключительно числа, то создадим колонку с разницей оставленных отзывов в днях, переведем их
# в формат integer

df['Reviews1'] = df['Reviews'].apply(lambda x: x[0] if len(x)>=1 else None)
df['Reviews2'] = df['Reviews'].apply(lambda x: x[1] if len(x)==2 else None)

df['Reviews1'] = pd.to_datetime(df['Reviews1'])
df['Reviews2'] = pd.to_datetime(df['Reviews2'])
df['Reviews_Dif'] = df['Reviews1'] - df['Reviews2']
df['Reviews_Dif'] = pd.to_numeric(df['Reviews_Dif'].dt.days, downcast='integer')

In [56]:
# Заменим NaN значения на среднее

df['Reviews_Dif'] = df['Reviews_Dif'].fillna(df['Reviews_Dif'].mean())

In [57]:
# Удалим ненформативнык колонки, в том числе и "Price Range"

df = df.drop(['Reviews', 
              'URL_TA', 'ID_TA', 'Reviews1',
             'Reviews2'], axis=1)

In [58]:
df.head()

,Restaurant_id,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Capital,Amsterdam,Athens,Barcelona,...,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich,Reviews_Dif
0,id_5569,8,5570.0,3.5,2.0,194.0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,41.000000
1,id_1535,1,1537.0,4.0,2.0,10.0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,382.000000
2,id_352,8,353.0,4.5,3.0,688.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.000000
3,id_3456,1,3458.0,5.0,2.0,3.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,124.088186
4,id_615,6,621.0,4.0,2.0,84.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,272.000000


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Cuisine Style      40000 non-null  int64  
 2   Ranking            40000 non-null  float64
 3   Rating             40000 non-null  float64
 4   Price Range        40000 non-null  float64
 5   Number of Reviews  40000 non-null  float64
 6   Capital            40000 non-null  int64  
 7   Amsterdam          40000 non-null  uint8  
 8   Athens             40000 non-null  uint8  
 9   Barcelona          40000 non-null  uint8  
 10  Berlin             40000 non-null  uint8  
 11  Bratislava         40000 non-null  uint8  
 12  Brussels           40000 non-null  uint8  
 13  Budapest           40000 non-null  uint8  
 14  Copenhagen         40000 non-null  uint8  
 15  Dublin             40000 non-null  uint8  
 16  Edinburgh          400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [60]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [61]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [62]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [63]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cuisine Style      40000 non-null  int64  
 1   Ranking            40000 non-null  float64
 2   Price Range        40000 non-null  float64
 3   Number of Reviews  40000 non-null  float64
 4   Capital            40000 non-null  int64  
 5   Amsterdam          40000 non-null  uint8  
 6   Athens             40000 non-null  uint8  
 7   Barcelona          40000 non-null  uint8  
 8   Berlin             40000 non-null  uint8  
 9   Bratislava         40000 non-null  uint8  
 10  Brussels           40000 non-null  uint8  
 11  Budapest           40000 non-null  uint8  
 12  Copenhagen         40000 non-null  uint8  
 13  Dublin             40000 non-null  uint8  
 14  Edinburgh          40000 non-null  uint8  
 15  Geneva             40000 non-null  uint8  
 16  Hamburg            400

# Создаём, обучаем и тестируем модель

In [64]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [65]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [66]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2118135
